# Rescan photos scripts

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate(r"C:\!Dev\projects\book_detection\keys\avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

db = firestore.client()

#### Not valid photos

In [5]:
res = db.collection('photos').where(u'valid', u'==', False).stream()
finded = set()
for doc in res:
    finded.add(doc.id)
print('not valid:', len(finded))
#finded

not valid: 15


#### Photos without block 'recognition_stats'

In [9]:
res = db.collection('photos').stream()
finded = set()
for doc in res:
    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    if 'recognition_stats' in data:
        continue
    finded.add(doc.id)

print('finded', len(finded))
#finded

finded 1241


#### Rescan one photo

In [20]:
%%time

import requests
photo_id = '0mdXOv2XMONaGUvyHHL5'
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()

res = requests.post(url, json={'photo_id': photo_id})
json = res.json()
print(res.status_code, res.reason)
print('duration:', json['duration'])
print('recognized:', len(json['recognized_books']))
for n, el in enumerate(json['recognized_books'], 1):
    print(f'{n}) {el}')

200 OK
duration: 0:00:44.981972
recognized: 14
1) 9785050040695 Современный американский детектив. Выпуск 1 Стаут Рекс Тодхантер, Макдональд Росс, Болл Джон
2) 9785871980514 Нужный образ Хоран Джеймс
3) 9785050027177 Дик Френсис. Кураж. Найо Марш.Занавес опускается Марш Найо, Фрэнсис Дик
4) 9785170909933 Крутая дамочка, или Нежнее чем польская панна Вильмонт Екатерина Николаевна
5) 9785265009937 Именем закона. Современный советский детектив. 1989 Карпович Ярослав, Булгакова Инна Валентиновна, Рябов Гелий Трофимович, Хруцкий Эдуард Анатольевич, Ромов Анатолий Сергеевич
6) 9785969407817 Психология эмоций : учебное пособие / А. С. Осипова, А. Н. Сударик, С. Н. Федотов ; Федеральное государственное казенное образовательное учреждение высшего образования "Московский университет Министерства внутренних дел Российской Федерации имени В. Я. Кикотя", Кафедра психологии учебно-научного комплекса психологии служебной деятельности. - Москва : Московский ун-т МВД России им. В. Я. Кикотя, 2019. - 14